<a href="https://colab.research.google.com/github/kasobol/Lab_3_sem_3/blob/master/AnalisDataTrades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Загрузка данных

import numpy as np
import sklearn
import gc
from sklearn.model_selection import train_test_split

data = []
with open("/content/drive/MyDrive/ColabNotebooks/Data_Trades/XBT_Price_Jump_Min(0.5)_Data_(2017_11_01).txt", 'r') as fObj:
  data = np.array(fObj.read().split()).astype(float)

#data = data[10000000:len(data)]
data = data[:]
x_data = np.array([data[i: i + 50] for i in range(0, len(data) - 54)])
y_data = np.array([np.sum(data[i: i + 5]) for i in range(50, len(data) - 4)])
#y_data = data[50:len(data)]

delimit = int(0.8 * len(x_data))

train_data = x_data[:delimit]
train_targets = y_data[:delimit]
test_data = x_data[delimit:]
test_targets = y_data[delimit:]

#train_data, test_data, train_targets, test_targets = train_test_split(x_data, y_data, test_size = 0.2, shuffle = True)
del x_data
del y_data
del data
gc.collect()


0

In [3]:
mas = np.abs(test_targets)
mas.mean()

21.68388689010385

In [4]:
# Нормализация данных

from sklearn import preprocessing

train_data = preprocessing.normalize(train_data)
test_data = preprocessing.normalize(test_data)

#mean = train_data.mean(axis=0)
#train_data -= mean
#std = train_data.std(axis=0)
#train_data /= std
#test_data -= mean
#test_data /= std

In [5]:
# Конструирование сети

from keras import models
from keras import layers
#from tensorflow.keras.optimizers import Adam

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(100, activation='relu',
  input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1))
  optimizer='rmsprop'
  #optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
  model.compile(optimizer = optimizer, loss='mae', metrics=['mae'])
  return model

In [6]:
test_data.shape

(2124624, 50)

In [7]:
test_targets.shape

(2124624,)

In [25]:
num_epochs = 20

model = build_model()

history = model.fit(train_data, train_targets,
 validation_data = (test_data, test_targets),
 epochs=num_epochs, batch_size = 256, verbose = 1)

Epoch 1/20
33198/33198 [==============================] - 87s 3ms/step - loss: 7.6864 - mae: 7.6864 - val_loss: 20.1663 - val_mae: 20.1663
Epoch 2/20
33198/33198 [==============================] - 88s 3ms/step - loss: 7.5260 - mae: 7.5260 - val_loss: 19.9397 - val_mae: 19.9397
Epoch 3/20
33198/33198 [==============================] - 87s 3ms/step - loss: 7.4500 - mae: 7.4500 - val_loss: 19.9152 - val_mae: 19.9152
Epoch 4/20
33198/33198 [==============================] - 81s 2ms/step - loss: 7.4027 - mae: 7.4027 - val_loss: 19.7809 - val_mae: 19.7809
Epoch 5/20
33198/33198 [==============================] - 82s 2ms/step - loss: 7.3764 - mae: 7.3764 - val_loss: 19.7649 - val_mae: 19.7649
Epoch 6/20
33198/33198 [==============================] - 86s 3ms/step - loss: 7.3502 - mae: 7.3502 - val_loss: 19.7058 - val_mae: 19.7058
Epoch 7/20
33198/33198 [==============================] - 85s 3ms/step - loss: 7.3314 - mae: 7.3314 - val_loss: 19.6804 - val_mae: 19.6804
Epoch 8/20
33198/33198 [===

In [ ]:
model.

In [27]:
mas = model(test_data)

In [28]:
test_targets

array([   3.5,  -12.5,    1.5, ...,  -84.5,  -71. , -101.5])

In [29]:
import tensorflow as tf
mas = tf.reshape(mas, [-1]).numpy()

In [30]:
mas

array([  9.505546 ,  -5.692021 ,   9.090295 , ..., -11.308599 ,
         9.948551 ,  -2.8652139], dtype=float32)

In [31]:
count1 = 0
all_count1 = len(mas)
for i in range(0, len(mas)):
  if mas[i] >= 0 and test_targets[i] >= 0:
    #all_count = all_count + 1
    count1 = count1 + 1
  elif mas[i] < 0 and test_targets[i] < 0:
    #all_count = all_count + 1
    count1 = count1 + 1

In [32]:
count1 / all_count1

0.7303970020107087

In [33]:
count = 0
all_count = 0
for i in range(0, len(mas)):
  if mas[i] >= 0:
    all_count = all_count + mas[i]
    count = count + test_targets[i]
  else:
    all_count = all_count - mas[i]
    count = count - test_targets[i]

In [34]:
count / all_count

2.1010603231450973